### **Предсказание биологического ответа молекул по их химическому составу**
#### Задание: ML-7. Прогнозирование биологическогго ответа (HW-3)

В качестве метрики используется F1-score.  
Предварительная обработка не требуется, данные закодированы и нормализованы.
  
Обучить две модели: **логистическую регрессию и случайный лес**.  
Сделать подбор гиперпараметров базовыми и продвинутми методами оптимизации.  
Использовать **все четыре метода**: *GridSearchCV, RandomizedSearchCV, Hyperopt, Optuna*.  
Максимальное количество итераций не должно превышать 50.

In [1]:
import pandas as pd
import numpy as np
from warnings import simplefilter
simplefilter(action="ignore", category=Warning)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials
import optuna
from scipy.stats import uniform

In [2]:
data = pd.read_csv('data/_train_sem09__1_.zip')
data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


###  Разделение набора данных на учебные и проверочные данные

In [3]:
# Целевой столбец Activity 
# Разделение на train и test в соотношении 80/20
X = data.drop(columns='Activity')
y = data['Activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, \
    test_size=0.2, stratify=y, random_state=42)

# Обучение двух моделей: логистическая регрессия и случайный лес
#### Логистическая регрессия

In [4]:
# Создание модели.  
log_reg = LogisticRegression(n_jobs=-1, random_state=42, max_iter=50)

# Обучение модели.  
log_reg.fit(X_train, y_train)
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)

# Вывод результатов.  
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

f1 тренировочная выборка: 0.8769
f1 тестовая выборка: 0.7767


#### Случайный лес

In [5]:
# Создание модели.  
rfс = RandomForestClassifier(n_jobs=-1, random_state=42)

# Обучение модели.  
rfс.fit(X_train, y_train)
y_train_pred = rfс.predict(X_train)
y_test_pred = rfс.predict(X_test)

# Вывод результатов.  
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

f1 тренировочная выборка: 1.0000
f1 тестовая выборка: 0.8048


# Базовые методы оптимизации с кросс-валидацией

### **GridSearchCV**
-----------------------------------------
#### Логистическая регрессия

In [6]:
# Сетка гиперпараметров.  
param_grid = [
    {'solver': ['saga'],
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.05, 0.1, 0.5, 1.]},
    {'solver': ['lbfgs'],
    'penalty': ['l2'],
    'C': [0.01, 0.05, 0.1, 0.5, 1.]}
]

# Поиск гиперпараметров с кросс-валидацией.  
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=50),
    param_grid=param_grid,
    scoring = 'f1',
    cv=5, # Количество фолдов
    n_jobs = -1
)  

# Обучение модели.  
grid_search.fit(X_train, y_train) 
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

In [7]:
# Вывод результатов.  
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

f1 тренировочная выборка: 0.8433
f1 тестовая выборка: 0.7825
Наилучшие значения гиперпараметров: {'C': 0.05, 'penalty': 'l2', 'solver': 'lbfgs'}


#### Случайный лес

In [8]:
# Сетка гиперпараметров
param_grid = [
    {'n_estimators': [80, 200, 300],
     'criterion': ['gini', 'entropy'],
     'min_samples_leaf': [5, 20, 50],
     'max_depth': [10, 30, 50]
    }
]

# Поиск гиперпараметров с кросс-валидацией.  
grid_search = GridSearchCV(
    estimator = RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    scoring = 'f1',
    cv=5, # Количество фолдов.  
    n_jobs = -1
)  

# Обучение модели.  
grid_search.fit(X_train, y_train) 
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

In [9]:
# Вывод результатов
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))
print('Наилучшие значения гиперпараметров: {}'.format(grid_search.best_params_))

f1 тренировочная выборка: 0.9554
f1 тестовая выборка: 0.8058
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 5, 'n_estimators': 300}


### **RandomizedSearchCV**
---------------------------------------
#### Логистическая регрессия

In [10]:
# Сетка гиперпараметров.  
param_grid = [
    {'solver': ['saga'],
    'penalty': ['l1', 'l2'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))},
    {'solver': ['lbfgs'],
    'penalty': ['l2'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}
]

# Случайный поиск гиперпараметров с кросс-валидацией.  
random_search = RandomizedSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_grid, 
    scoring = 'f1',
    cv=5,
    n_iter=30, 
    n_jobs = -1
)  

# Обучение модели
random_search.fit(X_train, y_train) 
y_train_pred = random_search.predict(X_train)
y_test_pred = random_search.predict(X_test)

In [11]:
# Вывод результатов.  
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))
print('Наилучшие значения гиперпараметров: {}'.format(random_search.best_params_))

f1 тренировочная выборка: 0.8355
f1 тестовая выборка: 0.7826
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.34}


#### Случайный лес

In [12]:
# Сетка гиперпараметров
param_grid = [
    {'n_estimators': [80, 200, 30],
     'criterion': ['gini', 'entropy'],
     'min_samples_leaf': [5, 20, 50],
     'max_depth': [10, 30, 50]
    }
]

# Поиск гиперпараметров с кросс-валидацией.  
random_search = RandomizedSearchCV(
    estimator = RandomForestClassifier(random_state=42), 
    param_distributions=param_grid, 
    scoring = 'f1',
    cv=5, 
    n_iter = 30, 
    n_jobs = -1
)  

# Обучение модели.  
random_search.fit(X_train, y_train) 
y_train_pred = random_search.predict(X_train)
y_test_pred = random_search.predict(X_test)

In [13]:
# Вывод результатов.  
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))
print('Наилучшие значения гиперпараметров: {}'.format(random_search.best_params_))

f1 тренировочная выборка: 0.9540
f1 тестовая выборка: 0.7985
Наилучшие значения гиперпараметров: {'n_estimators': 200, 'min_samples_leaf': 5, 'max_depth': 50, 'criterion': 'entropy'}


# Продвинутые методы оптимизации с кросс-валидацией

### **Hyperopt**
-----------------------------
#### Логистическая регрессия

In [14]:
# Пространство гиперпараметров.  
space = hp.choice('parameter_combinations', 
                  [
                   {'solver': 'saga', 
                    'penalty': hp.choice('penalty', ['l1', 'l2']), 
                    'C': hp.uniform('C_saga', 0.01, 1)}, 
                   
                   {'solver': 'lbfgs', 
                    'penalty': 'l2', 
                    'C': hp.uniform('C_lbfgs', 0.01, 1)}
                   ]
)

# Целевая функция.  
def obj_func(params, cv=5, X=X_train, y=y_train, random_state=42):
    params = {'solver': params['solver'], 
              'penalty': params['penalty'], 
              'C': params['C']
             }
      
    model = LogisticRegression(**params, class_weight='balanced', 
        random_state=random_state, max_iter=50
    )
      
    # Обучаем модель с помощью кросс-валидации.  
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()

    return -score 

trials = Trials()
best=fmin(obj_func, 
          space=space, 
          algo=tpe.suggest, 
          max_evals=20, 
          trials=trials, 
          rstate=np.random.default_rng(42)
         )
print(f'Наилучшие значения гиперпараметров: {best}')


100%|██████████| 20/20 [00:48<00:00,  2.42s/trial, best loss: -0.778377320521217]
Наилучшие значения гиперпараметров: {'C_saga': 0.10567819922023905, 'parameter_combinations': 0, 'penalty': 1}


In [15]:
# Расчет метрики для лучших найденных гиперпараметров.  
best_params = {'solver': 'saga',
          'penalty': 'l2', 
          'C': 0.10567819922023905
}

# Модель с лучшими параметрами.  
hyperopt_lr = LogisticRegression(**best_params, class_weight='balanced',
                                   random_state=42, max_iter=50
)
hyperopt_lr.fit(X_train, y_train)
y_train_pred = hyperopt_lr.predict(X_train)
y_test_pred = hyperopt_lr.predict(X_test)

In [16]:
best_score = (np.array(list(x['result']['loss'] 
                            for x in trials.trials)) * (-1)).max()

print('f1 кросс-валидация: {:.4f}'.format(best_score))
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

f1 кросс-валидация: 0.7784
f1 тренировочная выборка: 0.8509
f1 тестовая выборка: 0.7844


#### Случайный лес

In [17]:
# Пространство гиреопараметров.  
space={'n_estimators': hp.quniform('n_estimators', 100, 500, 1),
       'criterion': hp.choice('criterion', ['gini', 'entropy']),
       'max_depth' : hp.quniform('max_depth', 1, 100, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 100, 1)
      }


# Целевая функция.  
def obj_func(params, cv=5, X=X_train, y=y_train, random_state=42):
    params = {'n_estimators': int(params['n_estimators']),
              'criterion': params['criterion'],
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf'])
             }
  
    model = RandomForestClassifier(**params, class_weight='balanced', 
                                   n_jobs=-1, random_state=random_state)
    
    # Обучаем модель с помощью кросс-валидации.  
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()
    
    return -score

trials = Trials()
best=fmin(obj_func, 
          space=space, 
          algo=tpe.suggest, 
          max_evals=20, 
          trials=trials, 
          rstate=np.random.default_rng(42)
         )
print(f'Наилучшие значения гиперпараметров: {best}')

100%|██████████| 20/20 [00:43<00:00,  2.17s/trial, best loss: -0.8003945912298767]
Наилучшие значения гиперпараметров: {'criterion': 0, 'max_depth': 28.0, 'min_samples_leaf': 6.0, 'n_estimators': 114.0}


In [18]:
# Расчет метрики для лучших найденных гиперпараметров.  
best_score = (np.array(list(x['result']['loss'] 
                            for x in trials.trials)) * (-1)).max()

best_params = {
    'n_estimators': int(best['n_estimators']),
    'criterion': 'gini',
    'max_depth': int(best['max_depth']),
    'min_samples_leaf': int(best['min_samples_leaf'])
}

# Модель с лучшими параметрами.  
hyperopt_rf = RandomForestClassifier(**best_params, class_weight='balanced',
                                 n_jobs=-1, random_state=42)
hyperopt_rf.fit(X_train, y_train)
y_train_pred = hyperopt_rf.predict(X_train)
y_test_pred = hyperopt_rf.predict(X_test)
print('f1 кросс-валидация: {:.4f}'.format(best_score))
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

f1 кросс-валидация: 0.8004
f1 тренировочная выборка: 0.9262
f1 тестовая выборка: 0.7856


### **Optuna**
----------------------
#### Логистическая регрессия

In [19]:
# Целевая функция
def obj_func(trial):
    # пространство гиперпараметров
    solver = trial.suggest_categorical('solver', ['saga', 'lbfgs'])
    if solver == 'saga':
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    else:
        penalty = 'l2'
    C = trial.suggest_uniform('C', 0.01, 1)
    
    model = LogisticRegression(
        solver=solver,
        penalty=penalty,
        C=C,
        class_weight='balanced', 
        random_state=42, 
        max_iter=50
    )    
    
    # Обучаем модель с помощью кросс-валидации    
    score = cross_val_score(
        model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
    
    return score

# Поиск оптимальных гиперпараметров
sampler = optuna.samplers.TPESampler(seed=42)
study_lr = optuna.create_study(
    sampler=sampler, study_name='LogisticRegression', direction='maximize')
study_lr.optimize(obj_func, n_trials=20)


# Модель с лучшими параметрами
optuna_lr = LogisticRegression(
    **study_lr.best_params, class_weight='balanced', n_jobs=-1,
    random_state=42, max_iter=50
)
optuna_lr.fit(X_train, y_train)
y_train_pred = optuna_lr.predict(X_train)
y_test_pred = optuna_lr.predict(X_test)

[I 2022-07-17 17:44:31,013] A new study created in memory with name: LogisticRegression
[I 2022-07-17 17:44:32,302] Trial 0 finished with value: 0.7681021300072938 and parameters: {'solver': 'lbfgs', 'C': 0.7346740023932911}. Best is trial 0 with value: 0.7681021300072938.
[I 2022-07-17 17:44:36,974] Trial 1 finished with value: 0.7715489215295548 and parameters: {'solver': 'saga', 'penalty': 'l1', 'C': 0.8675143843171859}. Best is trial 1 with value: 0.7715489215295548.
[I 2022-07-17 17:44:38,393] Trial 2 finished with value: 0.7813910236649433 and parameters: {'solver': 'lbfgs', 'C': 0.03037864935284442}. Best is trial 2 with value: 0.7813910236649433.
[I 2022-07-17 17:44:42,812] Trial 3 finished with value: 0.7841523028368709 and parameters: {'solver': 'saga', 'penalty': 'l1', 'C': 0.1915704647548995}. Best is trial 3 with value: 0.7841523028368709.
[I 2022-07-17 17:44:44,118] Trial 4 finished with value: 0.7733707879246805 and parameters: {'solver': 'lbfgs', 'C': 0.4376255684556946

In [20]:
print('Наилучшие значения гиперпараметров: {}'.format(study_lr.best_params))
print('f1 кросс-валидация: {:.4f}'.format(study_lr.best_value))
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.24731945619521178}
f1 кросс-валидация: 0.7859
f1 тренировочная выборка: 0.8255
f1 тестовая выборка: 0.7752


#### Случайный лес

In [21]:
# Целевая функция
def obj_func(trial):
    # пространство гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 500, 1)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 1, 100, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 100, 1)
  
    # модель
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        class_weight='balanced', n_jobs=-1,
        random_state=42
    )
  
    # Обучаем модель с помощью кросс-валидации    
    score = cross_val_score(
        model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
    
    return score

# Поиск оптимальных гиперпараметров
sampler = optuna.samplers.TPESampler(seed=42)
study_rf = optuna.create_study(
    sampler=sampler, study_name='RandomForestClassifier', direction='maximize')
study_rf.optimize(obj_func, n_trials=20)


# Модель с лучшими параметрами  
optuna_rf = RandomForestClassifier(
    **study_rf.best_params, class_weight='balanced', n_jobs=-1,
    random_state=42
)
optuna_rf.fit(X_train, y_train)
y_train_pred = optuna_rf.predict(X_train)
y_test_pred = optuna_rf.predict(X_test)

[I 2022-07-17 17:45:41,349] A new study created in memory with name: RandomForestClassifier
[I 2022-07-17 17:45:43,786] Trial 0 finished with value: 0.7740914876214111 and parameters: {'n_estimators': 250, 'criterion': 'gini', 'max_depth': 60, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.7740914876214111.
[I 2022-07-17 17:45:45,088] Trial 1 finished with value: 0.73056530178717 and parameters: {'n_estimators': 162, 'criterion': 'entropy', 'max_depth': 61, 'min_samples_leaf': 72}. Best is trial 0 with value: 0.7740914876214111.
[I 2022-07-17 17:45:46,464] Trial 2 finished with value: 0.7686941932476204 and parameters: {'n_estimators': 108, 'criterion': 'gini', 'max_depth': 22, 'min_samples_leaf': 20}. Best is trial 0 with value: 0.7740914876214111.
[I 2022-07-17 17:45:48,240] Trial 3 finished with value: 0.76194474106495 and parameters: {'n_estimators': 173, 'criterion': 'entropy', 'max_depth': 44, 'min_samples_leaf': 30}. Best is trial 0 with value: 0.7740914876214111.
[I 202

In [22]:
print('Наилучшие значения гиперпараметров: {}'.format(study_rf.best_params))
print('f1 кросс-валидация: {:.4f}'.format(study_rf.best_value))
print('f1 тренировочная выборка: {:.4f}'.format(f1_score(y_train, y_train_pred)))
print('f1 тестовая выборка: {:.4f}'.format(f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров: {'n_estimators': 245, 'criterion': 'gini', 'max_depth': 78, 'min_samples_leaf': 2}
f1 кросс-валидация: 0.8203
f1 тренировочная выборка: 0.9929
f1 тестовая выборка: 0.8073
